In [1]:
from transformers import BertForSequenceClassification, BertTokenizer

model = BertForSequenceClassification.from_pretrained('boapps/kmdb_classification_model').to('cuda')
tokenizer = BertTokenizer.from_pretrained('SZTAKI-HLT/hubert-base-cc')

In [2]:
from datasets import load_dataset
from datasets import concatenate_datasets

dataset = load_dataset("boapps/kmdb_classification")
test_neg = dataset['test'].filter(lambda row: row['label'] == 0)
test_pos = dataset['test'].filter(lambda row: row['label'] == 1).select(range(len(test_neg)//10))
test_set = concatenate_datasets([test_pos, test_neg]).shuffle(seed=42)

<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow._fs.FileInfo size changed, may indicate binary incompatibility. Expected 64 from C header, got 88 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow._fs.FileSelector size changed, may indicate binary incompatibility. Expected 48 from C header, got 72 from PyObject
Found cached dataset parquet (/home/boa/.cache/huggingface/datasets/boapps___parquet/boapps--kmdb_classification-4003d65da9c3e34a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/boa/.cache/huggingface/datasets/boapps___parquet/boapps--kmdb_classification-4003d65da9c3e34a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-5c0dc0418152b777.arrow
Loading cached processed dataset at /home/boa/.cache/huggingface/datasets/boapps___parquet/boapps--kmdb_classification-4003d65da9c3e34a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-80ad540b3581070d.arrow
Loading cached shuffled indices for dataset at /home/boa/.cache/huggingface/datasets/boapps___parquet/boapps--kmdb_classification-4003d65da9c3e34a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-ee948041db649a27.arrow


In [3]:
test_set = test_set.map(lambda row: {'td': row['title']+'\n'+row['description']})
def tokenize_function(examples):
    return tokenizer(examples["td"], padding="max_length", truncation=True, max_length=512)
test_set = test_set.map(tokenize_function, batched=True)

Loading cached processed dataset at /home/boa/.cache/huggingface/datasets/boapps___parquet/boapps--kmdb_classification-4003d65da9c3e34a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-1b79992d93c60276.arrow


Map:   0%|          | 0/2084 [00:00<?, ? examples/s]

In [5]:
import torch
import torch.nn.functional as F
from tqdm import tqdm

tp = 0
tn = 0
fp = 0
fn = 0

with torch.no_grad():
    for a in tqdm(test_set):
        inputs = tokenizer(a['td'], return_tensors="pt").to('cuda')
        logits = model(**inputs).logits
        probabilities = F.softmax(logits[0], dim=-1)
        res = torch.argmax(logits, axis=1)[0]
        neg = probabilities[0] > 0.42
        if neg and a['label'] == 0:
            tn += 1
        elif not neg and a['label'] == 1:
            tp += 1
        elif not neg and a['label'] == 0:
            fp += 1
        elif neg and a['label'] == 1:
            fn += 1

print('precision', tp/(tp+fp))
print('recall', tp/(tp+fn))
print('accuracy', (tp+tn)/(tp+tn+fp+fn))

tp, tn, fp, fn

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2095/2095 [00:16<00:00, 126.85it/s]

precision 0.7392996108949417
recall 0.95
accuracy 0.9632458233890214


(190, 1828, 67, 10)

In [15]:
import numpy as np

np.array([tp, tn, fp, fn])*100/sum([tp, tn, fp, fn])

array([ 9.06921241, 87.25536993,  3.19809069,  0.47732697])